## P47 Wormhole 3 - get model confidences while training 
- Ok I might end up using these exports all the way back to like P41, we'll see
- First important thing to do is get wormhole idea working here. 

In [2]:
# ! pip install transformers matplotlib tqdm huggingface_hub

In [4]:
# from huggingface_hub import login
# login()

In [5]:
import torch
from transformers import pipeline
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import copy
from collections import OrderedDict

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import LlamaForCausalLM, PreTrainedTokenizerFast, LlamaConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

device='cuda'

In [6]:
model_id = "meta-llama/Llama-3.2-1B"
# model_id = "openai-community/gpt2"
# model_id = "google/gemma-3-1b-pt"
# model_id = "gpt2"

In [7]:
# #Only needed for llama random initialization
# config_dict = {
#   "_attn_implementation_autoset": True,
#   "architectures": [
#     "LlamaForCausalLM"
#   ],
#   "attention_bias": False,
#   "attention_dropout": 0.0,
#   "bos_token_id": 128000,
#   "eos_token_id": 128001,
#   "head_dim": 64,
#   "hidden_act": "silu",
#   "hidden_size": 2048,
#   "initializer_range": 0.02,
#   "intermediate_size": 8192,
#   "max_position_embeddings": 131072,
#   "mlp_bias": False,
#   "model_type": "llama",
#   "num_attention_heads": 32,
#   "num_hidden_layers": 16,
#   "num_key_value_heads": 8,
#   "pretraining_tp": 1,
#   "rms_norm_eps": 1e-05,
#   "rope_scaling": {
#     "factor": 32.0,
#     "high_freq_factor": 4.0,
#     "low_freq_factor": 1.0,
#     "original_max_position_embeddings": 8192,
#     "rope_type": "llama3"
#   },
#   "rope_theta": 500000.0,
#   "tie_word_embeddings": True,
#   "torch_dtype": "float32",
#   "transformers_version": "4.50.3",
#   "use_cache": True,
#   "vocab_size": 128256
# }

In [8]:
# Random init
# model_config = GPT2Config() #Full sized model
# model = GPT2LMHeadModel(model_config).to(device) #Ok i should see what happens with full and pretrained model. 
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# # Llama with random initialization
# model_config  = LlamaConfig.from_dict(config_dict)
# model = LlamaForCausalLM(model_config).to(device) 
# tokenizer = AutoTokenizer.from_pretrained(model_id)

#Pretrained
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model.eval();

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [7]:
# save_dir='/workspace/apr_24_9'

In [9]:
text = "The capital of France is Paris"
inputs = tokenizer(text, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]

In [10]:
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)

my_probs=F.softmax(outputs.logits, dim=-1)
y_one_hot=F.one_hot(input_ids, num_classes=model.config.vocab_size)
correct_next_token_probs = (my_probs[:,:-1]*y_one_hot[:,1:]).sum(-1) #I'm sure there's waaay more efficient ways to do this
my_loss=-torch.log(correct_next_token_probs).mean()
print(my_loss.item(), outputs.loss.item())

3.3751845359802246 3.3751840591430664


In [11]:
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)

my_probs=F.softmax(outputs.logits, dim=-1)
y_one_hot=F.one_hot(input_ids, num_classes=model.config.vocab_size)
correct_next_token_probs = (my_probs[:,:-1]*y_one_hot[:,1:]).sum(-1) #I'm sure there's waaay more efficient ways to do this
my_loss=-torch.log(correct_next_token_probs).mean()

paris_only_loss=-np.log(my_probs[0, 5, 12366].item())
print(my_loss.item(), outputs.loss.item(), paris_only_loss)

3.3751845359802246 3.3751840591430664 0.9376922065287221


In [14]:
my_probs.shape

torch.Size([1, 7, 128256])

In [15]:
sI=np.argsort(my_probs[0,5, :].detach().cpu().float().numpy())[::-1]

for i in sI[:10]:
    print(i, round(my_probs[0, 5, i].item(),5), tokenizer.decode([i]))

12366 0.39153  Paris
264 0.08419  a
279 0.0704  the
832 0.03096  one
1101 0.03061  also
2162 0.02528  home
3967 0.02462  known
539 0.01659  not
459 0.01241  an
7559 0.01172  located


In [16]:
def get_random_directions(params, seed=None):
    """
    Generate random direction vectors for each parameter tensor.
    
    Args:
        params: List of (name, parameter) tuples from model.named_parameters()
        seed: Random seed for reproducibility
        
    Returns:
        direction: OrderedDict mapping parameter names to random direction tensors
    """
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)
    
    direction = OrderedDict()
    for name, param in params:
        if param.requires_grad:
            direction[name] = torch.randn_like(param.data)
    
    return direction

In [17]:
def normalize_direction(direction, params):
    """
    Normalize the direction tensors to match the norm of each parameter tensor.
    
    Args:
        direction: OrderedDict mapping parameter names to direction tensors
        params: List of (name, parameter) tuples from model.named_parameters()
        
    Returns:
        normalized_direction: OrderedDict with normalized direction tensors
    """
    param_dict = OrderedDict(params)
    normalized_direction = OrderedDict()
    
    for name, dir_tensor in direction.items():
        param_norm = torch.norm(param_dict[name].data)
        dir_norm = torch.norm(dir_tensor)
        
        # Avoid division by zero
        if dir_norm > 0:
            normalized_direction[name] = dir_tensor * (param_norm / dir_norm)
        else:
            normalized_direction[name] = dir_tensor
    
    return normalized_direction

### Run once without Training

In [18]:
# output_dir='/home/stephen/backparopagation/apr_24_9'
output_dir='/workspace/apr_25_3'

In [19]:
prefix='pretrained_'
filtered_params = [(name, p) for name, p in model.named_parameters() if p.requires_grad]
# layers_name='all'

layers_name='first_8'
filtered_params = filtered_params[1:73] #First 8 layers - I like this - facorite so far

# layers_name='last_8'
# filtered_params = filtered_params[73:] #Last 8 layers - some nice structue, but yeah more parabolic than I would like

num_points=512 #CHANGE
random_seed_1=11
random_seed_2=111

# Generate and normalize random directions
direction1 = get_random_directions(filtered_params, seed=random_seed_1)
direction2 = get_random_directions(filtered_params, seed=random_seed_2)

direction1 = normalize_direction(direction1, filtered_params)
direction2 = normalize_direction(direction2, filtered_params)

original_params = OrderedDict()
for name, param in filtered_params:
    original_params[name] = param.data.clone()

alphas=np.linspace(-2.5, 2.5, num_points)
betas=np.linspace(-2.5, 2.5, num_points)

## Ok now let's get wormhol-ey

- Ok I'm confused, isn't the whole picture going to change as soon as I change my starting point? Or no?
- Ok maybe not in kinda na interesting way -> so if I move to -1, -1 for example, doesn't that just mean I actually need
- to change my testing range from to -1.5 to 3.5 or whatever, and then it should look the same?
- I think that makese sense -> let me noodle for a minutes
- Man I hope the conclusion i drew earlier makes some sense lol - this stuff gets tricky!
- Ok yep that's what happened -> make sense -> these should be fine -> i'll just change my alpha/beta range. 

In [21]:
n_steps=128 #TODO -> CHANGE to a big number
trajectory = []
lr=2e-6

os.makedirs(output_dir, exist_ok=True)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.SGD(model.parameters(), lr=lr) #Should try this

In [22]:
#Move away from center
# alpha_shift=-0.9 #This will probably take some trial and error
# beta_shift=0.05
alpha_shift=-0.9 #This will probably take some trial and error
beta_shift=0.05


alphas_shifted=alphas-alpha_shift
betas_shifted=betas-beta_shift

for name, param in model.named_parameters():
    if name in direction1 and name in direction2:
        param.data = original_params[name] + alpha_shift * direction1[name] + beta_shift * direction2[name]

original_params_shifted = OrderedDict()
for name, param in filtered_params:
    original_params_shifted[name] = param.data.clone()

In [23]:
model_outputs=[]
for step in tqdm(range(n_steps)):

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        my_probs=F.softmax(outputs.logits, dim=-1)
        # y_one_hot=F.one_hot(input_ids, num_classes=model.config.vocab_size)
        # correct_next_token_probs = (my_probs[:,:-1]*y_one_hot[:,1:]).sum(-1) #I'm sure there's waaay more efficient ways to do this
        # my_loss=-torch.log(correct_next_token_probs).mean()
        # paris_only_loss=-np.log(my_probs[0, 5, 12366].item())
        # print(my_loss.item(), outputs.loss.item(), paris_only_loss)
        sI=np.argsort(my_probs[0,5, :].detach().cpu().float().numpy())[::-1]
        current_outs=[]
        for i in sI[:10]:
            current_outs.append([i, round(my_probs[0, 5, i].item(),5), tokenizer.decode([i])])
        model_outputs.append(current_outs)

    model.train()
    optimizer.zero_grad()
    outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss #Ok not just paris loss here -> not sure how much I'm worried about that
    loss.backward()
    optimizer.step()

In [25]:
model_outputs[0]

[[37180, 0.04863, 'adar'],
 [25107, 0.01659, ' mineral'],
 [47415, 0.01525, 'Pose'],
 [50561, 0.00561, ' dwarf'],
 [6200, 0.00511, 'gl'],
 [80830, 0.00508, ' Dwarf'],
 [22919, 0.00415, ' Adult'],
 [57403, 0.00394, ' poc'],
 [51473, 0.00364, ' Pose'],
 [43521, 0.00344, 'velt']]

In [27]:
model_outputs[1]

[[37180, 0.0235, 'adar'],
 [11, 0.01985, ','],
 [25107, 0.01922, ' mineral'],
 [12140, 0.01128, ' Louis'],
 [4701, 0.01003, ' Ag'],
 [47415, 0.00842, 'Pose'],
 [50561, 0.00548, ' dwarf'],
 [30976, 0.00395, 'inet'],
 [50416, 0.00379, ' Mineral'],
 [22919, 0.00376, ' Adult']]

In [28]:
model_outputs[2]

[[12366, 0.17303, ' Paris'],
 [11, 0.10821, ','],
 [315, 0.01774, ' of'],
 [60704, 0.01544, 'Paris'],
 [12140, 0.01406, ' Louis'],
 [304, 0.01241, ' in'],
 [9822, 0.01167, ' France'],
 [4701, 0.01122, ' Ag'],
 [220, 0.0101, ' '],
 [13, 0.00681, '.']]

In [29]:
model_outputs[3]

[[12366, 0.81609, ' Paris'],
 [60704, 0.04212, 'Paris'],
 [11, 0.0167, ','],
 [9822, 0.01137, ' France'],
 [315, 0.00877, ' of'],
 [8753, 0.00305, ' French'],
 [304, 0.00304, ' in'],
 [12140, 0.00249, ' Louis'],
 [320, 0.00215, ' ('],
 [41958, 0.00193, ' paris']]

In [30]:
model_outputs[4]

[[12366, 0.92809, ' Paris'],
 [60704, 0.03539, 'Paris'],
 [9822, 0.00653, ' France'],
 [315, 0.00449, ' of'],
 [11, 0.00232, ','],
 [8753, 0.0014, ' French'],
 [41958, 0.00112, ' paris'],
 [304, 0.00084, ' in'],
 [320, 0.00072, ' ('],
 [50100, 0.00065, 'France']]

In [33]:
model_outputs[10]

[[12366, 0.89602, ' Paris'],
 [374, 0.07165, ' is'],
 [60704, 0.01212, 'Paris'],
 [315, 0.00628, ' of'],
 [9822, 0.00352, ' France'],
 [41958, 0.00075, ' paris'],
 [320, 0.00073, ' ('],
 [574, 0.00069, ' was'],
 [8753, 0.0006, ' French'],
 [304, 0.00051, ' in']]

In [36]:
model_outputs[11]

[[12366, 0.91165, ' Paris'],
 [374, 0.06264, ' is'],
 [60704, 0.01071, 'Paris'],
 [315, 0.00425, ' of'],
 [9822, 0.00303, ' France'],
 [41958, 0.0007, ' paris'],
 [320, 0.00053, ' ('],
 [574, 0.00047, ' was'],
 [8753, 0.00046, ' French'],
 [304, 0.00033, ' in']]

In [37]:
model_outputs[12]

[[12366, 0.93694, ' Paris'],
 [374, 0.0431, ' is'],
 [60704, 0.00964, 'Paris'],
 [315, 0.00258, ' of'],
 [9822, 0.00248, ' France'],
 [41958, 0.00064, ' paris'],
 [320, 0.00033, ' ('],
 [8753, 0.00033, ' French'],
 [574, 0.00028, ' was'],
 [477, 0.0002, ' or']]

In [38]:
model_outputs[13]

[[12366, 0.95851, ' Paris'],
 [374, 0.02583, ' is'],
 [60704, 0.00873, 'Paris'],
 [9822, 0.00197, ' France'],
 [315, 0.00148, ' of'],
 [41958, 0.00057, ' paris'],
 [8753, 0.00023, ' French'],
 [320, 0.0002, ' ('],
 [574, 0.00016, ' was'],
 [477, 0.00012, ' or']]

In [39]:
model_outputs[15]

[[12366, 0.98119, ' Paris'],
 [374, 0.00832, ' is'],
 [60704, 0.0072, 'Paris'],
 [9822, 0.00119, ' France'],
 [315, 0.00051, ' of'],
 [41958, 0.00044, ' paris'],
 [8753, 0.00011, ' French'],
 [320, 7e-05, ' ('],
 [574, 5e-05, ' was'],
 [477, 4e-05, ' or']]

In [40]:
model_outputs[16]

[[12366, 0.98622, ' Paris'],
 [60704, 0.00656, 'Paris'],
 [374, 0.00483, ' is'],
 [9822, 0.00093, ' France'],
 [41958, 0.00039, ' paris'],
 [315, 0.00031, ' of'],
 [8753, 7e-05, ' French'],
 [320, 4e-05, ' ('],
 [574, 3e-05, ' was'],
 [477, 2e-05, ' or']]

In [41]:
model_outputs[18]

[[12366, 0.99127, ' Paris'],
 [60704, 0.00553, 'Paris'],
 [374, 0.00182, ' is'],
 [9822, 0.00058, ' France'],
 [41958, 0.00031, ' paris'],
 [315, 0.00013, ' of'],
 [8753, 4e-05, ' French'],
 [320, 2e-05, ' ('],
 [574, 1e-05, ' was'],
 [477, 1e-05, ' or']]

In [42]:
model_outputs[:50]

[[[37180, 0.04863, 'adar'],
  [25107, 0.01659, ' mineral'],
  [47415, 0.01525, 'Pose'],
  [50561, 0.00561, ' dwarf'],
  [6200, 0.00511, 'gl'],
  [80830, 0.00508, ' Dwarf'],
  [22919, 0.00415, ' Adult'],
  [57403, 0.00394, ' poc'],
  [51473, 0.00364, ' Pose'],
  [43521, 0.00344, 'velt']],
 [[37180, 0.0235, 'adar'],
  [11, 0.01985, ','],
  [25107, 0.01922, ' mineral'],
  [12140, 0.01128, ' Louis'],
  [4701, 0.01003, ' Ag'],
  [47415, 0.00842, 'Pose'],
  [50561, 0.00548, ' dwarf'],
  [30976, 0.00395, 'inet'],
  [50416, 0.00379, ' Mineral'],
  [22919, 0.00376, ' Adult']],
 [[12366, 0.17303, ' Paris'],
  [11, 0.10821, ','],
  [315, 0.01774, ' of'],
  [60704, 0.01544, 'Paris'],
  [12140, 0.01406, ' Louis'],
  [304, 0.01241, ' in'],
  [9822, 0.01167, ' France'],
  [4701, 0.01122, ' Ag'],
  [220, 0.0101, ' '],
  [13, 0.00681, '.']],
 [[12366, 0.81609, ' Paris'],
  [60704, 0.04212, 'Paris'],
  [11, 0.0167, ','],
  [9822, 0.01137, ' France'],
  [315, 0.00877, ' of'],
  [8753, 0.00305, ' French']

In [17]:
# alphas_shifted

In [18]:
# betas_shifted

In [ ]:
for step in range(n_steps):
    if step>10: #Previous 11 steps already rendered!
        losses=[]
        model.eval();
        with torch.no_grad():
            for i, alpha in enumerate(tqdm(alphas_shifted)):
                losses.append([])
                for j, beta in enumerate(betas_shifted):
                    for name, param in model.named_parameters():
                        if name in direction1:
                            param.data = original_params_shifted[name] + alpha * direction1[name] + beta*direction2[name]
                    
                    outputs = model(input_ids, labels=input_ids)
                    my_probs=F.softmax(outputs.logits, dim=-1)
                    paris_only_loss=-np.log(my_probs[0, 5, 12366].item())
                    losses[-1].append(paris_only_loss)
            
            for name, param in model.named_parameters(): # Restore original shifted parameters
                if name in original_params: 
                    param.data.copy_(original_params_shifted[name])
        losses=np.array(losses)
        np.save(output_dir +'/'+str(step).zfill(3), losses)
        
        plt.clf()
        fig, ax = plt.subplots(figsize=(10, 8))
        contourf = ax.contourf(alphas, betas, losses, 20, cmap='viridis', alpha=0.8)
        contour = ax.contour(alphas, betas, losses, 30, colors='white', linewidths=0.5)
        plt.scatter(beta_shift, alpha_shift, c='m')
        plt.savefig(output_dir +'/'+str(step).zfill(3)+'.png')

    model.train()
    optimizer.zero_grad()
    outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss #Ok not just paris loss here -> not sure how much I'm worried about that
    loss.backward()
    optimizer.step()

 75%|███████▍  | 382/512 [1:04:15<21:51, 10.09s/it]

In [ ]:
# alpha_shift=1 #This will probably take some trial and error
# beta_shift=-1

# plt.clf()
# fig, ax = plt.subplots(figsize=(10, 8))
# contourf = ax.contourf(alphas, betas, losses, 20, cmap='viridis', alpha=0.8)
# contour = ax.contour(alphas, betas, losses, 30, colors='white', linewidths=0.5)
# plt.scatter(beta_shift, alpha_shift, c='m')
# plt.savefig(output_dir +'/'+str(step).zfill(3)+'.png')


---

Ok cool 1e-5 is probably a pretty good learning rate